In [22]:
import pandas as pd
import numpy as np

In [23]:
train_df = pd.DataFrame(pd.read_csv('../resources/combined_data/train_unduplicated_labels.csv'))
test_df = pd.read_csv('../resources/combined_data/test_with_text.csv')
train_df.head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,text,all_labels
0,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,The Impact of Dual Enrollment on College Degre...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,section title what is this study about text th...,national education longitudinal study
1,2f26f645-3dec-485d-b68d-f013c9e05e60,Educational Attainment of High School Dropouts...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,section title november 2004 text dropping out ...,national education longitudinal study
2,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29,Differences in Outcomes for Female and Male St...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,section title differences in outcomes for fema...,national education longitudinal study
3,5c9a3bc9-41ba-4574-ad71-e25c1442c8af,Stepping Stone and Option Value in a Model of ...,National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,section title abstract text federal reserve ba...,national education longitudinal study
4,c754dec7-c5a3-4337-9892-c02158475064,"Parental Effort, School Resources, and Student...",National Education Longitudinal Study,National Education Longitudinal Study,national education longitudinal study,section title abstract text this article inves...,national education longitudinal study


In [24]:
X = train_df['text']
y = train_df['all_labels']
print(X.shape, y.shape)

(19661,) (19661,)


In [41]:
# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=42)

In [42]:
# more functions
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, confusion_matrix

def model_test(model):
    '''Quickly creates pipeline and displays scores for testing'''
    
    clf_pipe = Pipeline([
        ('vect', CountVectorizer(stop_words='english', max_df=0.7)),
        ('tfidf', TfidfTransformer()),
        ('clf', model),
    ])

    clf_pipe.fit(X_train, y_train)
    
    print(model)

    score = clf_pipe.score(X_test, y_test)
    print("Score:", score)
    
def create_submit(model):
    '''Fits training data and makes sumbission predictions'''
    clf_pipe = Pipeline([
        ('vect', CountVectorizer(stop_words='english', max_df=0.7)),
        ('tfidf', TfidfTransformer()),
        ('clf', model),
    ])
    # Fit and transform data 
    clf_pipe.fit(X_train, y_train)
    # Make predictions
    pred = clf_pipe.predict(test_df.text)
    # add dataset labels preds to submission DF
    test_df['PredictionString'] = pred
    submission_df = test_df.drop(columns=['text']).set_index('Id')
    return submission_df

In [44]:
from sklearn.linear_model import PassiveAggressiveClassifier 
clf = PassiveAggressiveClassifier(validation_fraction=.9, warm_start=True)
model_test(clf)

# Duplicates:
# Score: 0.779

# .1 testing:
# 0.794

# .8 val
# 0.798

# .9 val
# 0.799!

# .05 testing
# 0.796

# warm start + .1 testing
# 0.795

# warm start defult val
# 0.797

# test .15

PassiveAggressiveClassifier(validation_fraction=0.9, warm_start=True)
Score: 0.7935593220338983
